In [1]:
# !pip install transformers==4.5.0
# !pip install pytorch-lightning==1.2.7
# !pip install seaborn
# !pip install gdown

In [2]:
# !pip install wandb

In [3]:
# !pip install sklearn

In [21]:
import json
import pandas as pd
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import WandbLogger
from sklearn.model_selection import train_test_split
import textwrap

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
)
from tqdm.auto import tqdm

In [22]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10

In [23]:
pl.seed_everything(42)

Global seed set to 42


42

In [59]:
# !git clone https://gitlab.com/shimorina/webnlg-dataset.git

In [8]:
# mv 'webnlg-ataset/release_v3.0/en' 'webnlg_data'

In [5]:
import glob
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd

train_path = "train/**/*"
dev_path = "dev/**/*"
test_path = "test/rdf-to-text-generation-test-data-with-refs-en"
sets = [train_path, dev_path, test_path]

for s in sets:
    files = glob.glob(os.getcwd() + "/webnlg_data/" + s + ".xml", recursive=True)    
    inputs = list()
    outputs = list()
    for file in files:
        tree = ET.parse(file)
        root = tree.getroot()    
        for sub_root in root:
            for entry in sub_root:
                for element in entry:
                    if "modifiedtripleset" in str(element):
                        inp = " && ".join([triple.text for triple in element])
                        # inp = " && ".join(["|".join([e.strip("\"") for e in triple.text.split("|")]) for triple in element])
                    if "lex" in str(element):
                        out = element.text
                        inputs.append(inp)
                        outputs.append(out)

    mdata_dct={"input_text":[], "target_text":[]}
    for i, _ in enumerate(inputs):
        mdata_dct['input_text'].append(inputs[i])
        mdata_dct['target_text'].append(outputs[i])


    df=pd.DataFrame(mdata_dct)
    df.to_csv('webNLG2020_' + s.split("/")[0] + '.csv', index=False)

In [24]:
TRAIN_PATH = 'webNLG2020_train.csv'
DEV_PATH = 'webNLG2020_dev.csv'
TEST_PATH = 'webNLG2020_test.csv'

In [25]:
train_df = pd.read_csv(TRAIN_PATH)
dev_df = pd.read_csv(DEV_PATH)
test_df = pd.read_csv(TEST_PATH)

In [26]:
full_data = pd.concat([train_df, dev_df, test_df])
full_data.head()

,input_text,target_text
0,"Aarhus_Airport | cityServed | ""Aarhus, Denmark""","The Aarhus is the airport of Aarhus, Denmark."
1,"Aarhus_Airport | cityServed | ""Aarhus, Denmark""","Aarhus Airport serves the city of Aarhus, Denm..."
2,Aarhus_Airport | cityServed | Aarhus,Aarhus airport serves the city of Aarhus.
3,Aarhus_Airport | elevationAboveTheSeaLevel | 25.0,Aarhus Airport is 25 metres above sea level.
4,Aarhus_Airport | elevationAboveTheSeaLevel | 25.0,Aarhus airport is at an elevation of 25 metres...


In [8]:
print(train_df.shape, dev_df.shape, test_df.shape)

(35426, 2) (4464, 2) (5150, 2)


In [36]:
MODEL_NAME = "t5-small"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [34]:
rdf_lens = list()
summary_lens = list()
for idx, row in full_data.iterrows():
    rdf_encoding = tokenizer(row["input_text"])
    rdf_lens.append(len(rdf_encoding.tokens()))
    summary_encoding = tokenizer(row["target_text"])
    summary_lens.append(len(summary_encoding.tokens()))

In [35]:
print(max(rdf_lens),max(summary_lens))

263 147


In [17]:
!wandb login '47616056d2ebbf7dea86db60a5fc58145cd234fc'

wandb: Appending key for api.wandb.ai to your netrc file: /home/studio-lab-user/.netrc


In [18]:
class WebNLGDataset(Dataset):
    
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: T5Tokenizer,
        text_max_token_len: int = 263,
        summary_max_token_len: int = 147
    ):
        
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]
        
        text = data_row['input_text']
        
        text_encoding = tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )
        
        summary_encoding = tokenizer(
            data_row["target_text"],
            max_length=self.text_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )
        
        labels = summary_encoding["input_ids"]
        labels[labels == 0] = -100
        
        return dict(
            text=text,
            summary=data_row["target_text"],
            text_input_ids=text_encoding["input_ids"].flatten(),
            text_attention_mask=text_encoding["attention_mask"].flatten(),
            labels=labels.flatten(),
            labels_attention_mask=summary_encoding["attention_mask"].flatten()
        )

In [8]:
class WebNLGDataModule(pl.LightningDataModule):
    
    def __init__(
        self,
        train_df: pd.DataFrame,
        dev_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size: int = 8,
        text_max_token_len: int = 263,
        summary_max_token_len: int = 147
    ):
        
        super().__init__()
        
        self.train_df = train_df
        self.dev_df = dev_df
        self.test_df = test_df
        
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
        
    def setup(self, stage=None):
        self.train_dataset = WebNLGDataset(
            self.train_df,
            self.tokenizer,
            self.text_max_token_len,
            self.summary_max_token_len
        )
        self.test_dataset = WebNLGDataset(
            self.test_df,
            self.tokenizer,
            self.text_max_token_len,
            self.summary_max_token_len
        )
        self.dev_dataset = WebNLGDataset(
            self.dev_df,
            self.tokenizer,
            self.text_max_token_len,
            self.summary_max_token_len
        )
        
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=2
        )
        
    def val_dataloader(self):
        return DataLoader(
            self.dev_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2
        )

    def test_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2
        )

In [10]:
N_EPOCHS = 3
BATCH_SIZE = 8


data_module = WebNLGDataModule(train_df, test_df, dev_df, tokenizer, batch_size=BATCH_SIZE)

In [11]:
class WebNLGModel(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        
    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
        
        output = self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask
        )
        
        return output.loss, output.logits
    
    def training_step(self, batch, batch_idx):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]
        
        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels
        )
        
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]
        
        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels
        )
        
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        input_ids = batch["text_input_ids"]
        attention_mask = batch["text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]
        
        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels
        )
        
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.0001)

In [12]:
model = WebNLGModel()

In [15]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

logger = WandbLogger()

trainer = pl.Trainer(
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    max_epochs=N_EPOCHS,
    gpus=1,                                     ###############
    progress_bar_refresh_rate=30
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [ ]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pavl_os. Use `wandb login --relogin` to force relogin



  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)
/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 4428: val_loss reached 1.34267 (best 1.34267), saving model to "/home/studio-lab-user/sagemaker-studiolab-notebooks/checkpoints/best-checkpoint-v6.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Epoch 1, global step 8857: val_loss reached 1.31121 (best 1.31121), saving model to "/home/studio-lab-user/sagemaker-studiolab-notebooks/checkpoints/best-checkpoint-v6.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

In [16]:
trained_model = WebNLGModel.load_from_checkpoint(
   trainer.checkpoint_callback.best_model_path
)

trained_model.freeze()

IsADirectoryError: [Errno 21] Is a directory: '/home/studio-lab-user/sagemaker-studiolab-notebooks'

In [29]:
trainer.checkpoint_callback.best_model_path

'/home/studio-lab-user/sagemaker-studiolab-notebooks/checkpoints/best-checkpoint-v3.ckpt'

In [30]:
def summarize(text):
    text_encoding = tokenizer(
        text,
        max_length=263,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )
    
    generated_ids = trained_model.model.generate(
        input_ids=text_encoding["input_ids"],
        attention_mask=text_encoding["attention_mask"],
        max_length=147,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )
        
    preds = [
        tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for gen_id in generated_ids
    ]
        
    return "".join(preds)

In [31]:
sample_row = test_df.iloc[0]
text = sample_row["input_text"]
model_summary = summarize(text)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/generation_utils.py:1777: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


In [32]:
text

'EstÃ¡dio_Municipal_Coaracy_da_Mata_Fonseca | location | Arapiraca && AgremiaÃ§Ã£o_Sportiva_Arapiraquense | league | Campeonato_Brasileiro_SÃ©rie_C && Campeonato_Brasileiro_SÃ©rie_C | country | Brazil && AgremiaÃ§Ã£o_Sportiva_Arapiraquense | nickname | "\'\'Alvinegro" && AgremiaÃ§Ã£o_Sportiva_Arapiraquense | ground | EstÃ¡dio_Municipal_Coaracy_da_Mata_Fonseca'

In [34]:
sample_row["target_text"]

'EstÃ¡dio Municipal Coaracy da Mata Fonseca is the name of the ground of AgremiaÃ§Ã£o Sportiva Arapiraquense in Arapiraca. AgremiaÃ§Ã£o Sportiva Arapiraquense, nicknamed "Alvinegro", lay in the Campeonato Brasileiro SÃ©rie C league from Brazil.'

In [35]:
model_summary

'Estdio_Municipal_Coaracy_da_Mata_Fonseca | location | Arapiraca && Agremia£o_Sportiva_Arapiraquense | league | Campeonato_Brasileiro_Srie_C && Campeonato_Brasileiro_Srie_C | country | Brazil && Agremia£o_Sportiva_Arapira'

In [38]:
sample_row = test_df.iloc[2]
text = sample_row["input_text"]
model_summary = summarize(text)

In [39]:
text

'Nie_Haisheng | birthDate | 1964-10-13 && Nie_Haisheng | occupation | Fighter_pilot'

In [40]:
sample_row["target_text"]

'Nie Haisheng, born on October 13, 1964, worked as a fighter pilot.'

In [41]:
model_summary

'Nie_Haisheng | birthDate | 1964-10-13 && Nie_Haisheng | occupation | Fighter_pilot'